# Forward Kinematics: Moving Robots with Mathematics

## Introduction

This Notebook will give a short introduction of forward kinematics, transformation matrices, and how to apply these principles to revolute joints. As an exercise, you will be given the opportunity to implement the forward kinematics for the Openmanipulator-X yourself. 

## What is Forward Kinematics in Robotics?

In robotics, kinematics focuses on the motion of robots: given a set of joint angles, where is the robot's end effector (like a robot arm's gripper)?

**Forward kinematics** is the process of determining the position and orientation of the end effector given the joint angles and the robot's kinematic structure. This process is "forward" because we start from the joint angles (inputs) and calculate the end effector pose (output).

## OpenManipulator-X and Its Coordinate System

OpenManipulator-X is a robotic manipulator consisting of revolute joints and links. The base link of OpenManipulator-X is located at the origin of the robot's coordinate system, while the end effector is at the tip of the gripper.

In OpenManipulator-X, the coordinate system is defined as follows: 

- The Z-axis points upwards.
- The X-axis points forwards.
- The Y-axis points to the left.

This affects how we interpret the position and orientation of each joint and link.

## Revolute Joints and Transformation Matrices

A robot manipulator is composed of rigid bodies called **links** connected by **joints**. The joints allow relative motion between the links. One common type of joint is the **revolute joint**, which allows rotation around a single axis. In the Openmanipulator-X robot, the joints are revolute joints. Except for the gripper, each joint has a single degree of freedom (DOF), which means that it can rotate around a single axis. The gripper is a prismatic joint, which means that it can move along a single axis. For now we will focus on revolute joints.

Each joint in a robot has an associated **transformation matrix** that describes the joint's position and orientation relative to its parent link. For a revolute joint, this transformation includes a static part (based on the joint's position and orientation in the robot's design) and a variable part (based on the current joint angle).

The transformation matrix for a revolute joint can be represented in homogeneous coordinates as follows:

```
T = | R  P |
    | 0  1 |
```

where `R` is a 3x3 rotation matrix, and `P` is a 3x1 position vector. The rotation matrix represents the orientation of the joint, and the position vector represents the position of the joint in the parent link's coordinate frame. 

## Putting it All Together: Forward Kinematics

Once we have the transformation matrix for each joint, we can calculate the overall transformation from the base link to the end effector by multiplying these matrices together in the correct order. This overall transformation matrix gives us the position and orientation of the end effector in the base link's coordinate frame, which is the result of forward kinematics.

Here is a simple example in pseudocode for a robot with one revolute joint:

```pseudocode
# Joint position (translation vector)
P = [x, y, z]

# Joint static orientation (rotation matrix)
R_static = rpy(roll, pitch, yaw)

# Joint variable orientation (rotation matrix)
θ = current_joint_angle
R_variable = RotationMatrix(θ, axis)

# Combine static and variable rotations
R = R_static * R_variable

# Homogeneous transformation matrix
T = [R P; 0 0 0 1]

# The end effector's position and orientation is given by T
```

For multiple joints in a chain, we can calculate the overall transformation matrix by multiplying the matrices for each joint together in the correct order. For example, if we have three joints, we can calculate the overall transformation matrix as follows:

```pseudocode
T = T_1 * T_2 * T_3
```

where `T_1` is the transformation matrix for the first joint, `T_2` is the transformation matrix for the second joint, and `T_3` is the transformation matrix for the third joint.

## Getting the Rotation Matrix from a Joint Angle
The angle of each joint in a robot manipulator will affect the orientation part of the transformation matrix. For revolute joints, rotation will take place around a specified axis.

For a revolute joint in a robot, the joint angle is often represented as a rotation around one of the coordinate axes. Let's assume that we are dealing with a rotation about the z-axis, which is a common case. The rotation matrices for a rotation of θ degrees around the x-, -y, -z -axis are:

```
Rx(θ) = | 1     0       0  |
        | 0 cos(θ) -sin(θ) |
        | 0 sin(θ)  cos(θ) |

Ry(θ) = |  cos(θ) 0 sin(θ) |
        |   0     1   0    |
        | -sin(θ) 0 cos(θ) |

Rz(θ) = | cos(θ) -sin(θ) 0 |
        | sin(θ)  cos(θ) 0 |
        |   0       0    1 |
```

These matrix Ry(θ) represent a counterclockwise rotation by θ radians around the y-axis. When you are calculating the forward kinematics, you replace the rotation part of the transformation matrix for each joint with the appropriate (correspodning to the rotation axis) rotation matrix based on the current joint angle. This rotation matrix is then multiplied with the static transformation matrix (which includes any static offsets and rotations defined in the URDF) to get the final transformation matrix for that joint.

Here is an example in pseudocode for a revolute joint rotating about the z-axis:

```pseudocode
# Joint static orientation (rotation matrix)
R_variable = [cos(θ) -sin(θ) 0; sin(θ) cos(θ) 0; 0 0 1]
```

In this matrix:

- cos(θ) and sin(θ) define the rotation around the z-axis. They define how much the x- and y-components change with the rotation of θ around z.
- The bottom row [0 0 1] remains the same because we're rotating around the z-axis, so the z component doesn't change. 
- The rotation angle θ should be provided in radians.

## Understanding the Openmanipulator-X 

To find the corresponding transformation matrices for the Openmanipulator-X, you can refer to the URDF file of the robot. The URDF file describes the structure of the robot in terms of links, joints, and their properties. You can find the URDF file for the Openmanipulator-X in the `../open_manipulator_description/urdf/` directory.

In the URDF file, you can search for the "joint" descriptions of each joint to find the following information about each joint:
```xml
  <!-- Joint 1 -->
  <joint name="joint1" type="revolute">
    <parent link="link1"/>
    <child link="link2"/>
    <origin rpy="0 0 0" xyz="0.012 0.0 0.017"/>
    <axis xyz="0 0 1"/>
    <limit effort="1" lower="-2.827433388230814" upper="2.827433388230814" velocity="4.8"/>
  </joint>
```

We can see that the information for implementing the transformation matrix is given in the `<origin>` and `<axis>` tags. The `<origin>` tag provides the position of the joint in the parent link's coordinate frame, and the `<axis>` tag provides the axis of rotation for the joint.

## Introduction into Forward Kinematics with the Openmanipulator-X

Now it's your turn! Your task is to implement the forward kinematics for the Openmanipulator-X with 4 revolute joints. 

To calculate the foward kinematics for the OpenManipulator-X we first look how we would do it for the first link and then move on to the other Link. 
Under the wollowing link we find the URDF file for the OpenManipulator-X: https://github.com/ROBOTIS-GIT/open_manipulator/blob/master/open_manipulator_description/urdf/open_manipulator.urdf.xacro 

An URDF file is an XML file that describes the physical parameters of a robot. There are specific libraries that can parse URDF files and extract the robot's physical parameters. For this exercise we will extract the physical parameters of the OpenManipulator-X by hand using the information we have from RViz. 

In [ ]:
import Pkg
Pkg.generate("OpenMEnv") # generate a new package
Pkg.activate("OpenMEnv") # activate the package
Pkg.add("RigidBodyDynamics")
Pkg.add("MeshCatMechanisms")
Pkg.add("MeshCat")
Pkg.add("LinearAlgebra")
Pkg.add("StaticArrays")
Pkg.add("Rotations")

In [2]:
import Pkg
Pkg.activate("OpenMEnv")
using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using LinearAlgebra
using StaticArrays
using Rotations

  Activating project at `~/Library/Mobile Documents/com~apple~CloudDocs/Arbeit/Lectures/Vorlesung - Ingenieurwissenschaftliche Grundlagen 3 (Numerische Methoden)/auxme_numerik/Excercise/Exercise 4/OpenMEnv`


In [ ]:
# example for link 1 - joint 1 - Rotation around z-axis

# Joint position (translation vector)
P = [0.012, 0.0, 0.017]  # [x, y, z]

# Joint orientation (rotation matrix)
R_static = RotXYZ(roll=0.0, pitch=0.0, yaw=0.0)  # from Rotations.jl's - note that this results in a 3x3 identity matrix (no rotation)

# Joint variable orientation (rotation matrix)
θ = 0.5
R_variable = # TODO: Create a rotation matrix from the angle θ around the z-axis

# Combine static and variable rotations
R = # TODO: Combine the static and variable rotation matrices

# Homogeneous transformation matrix
T = [R P; 0 0 0 1]

Let's compare the our calculated position after the first joint with the position of the first link using RigidBodyDynamics:

In [ ]:
srcdir = "../open_manipulator_description/urdf/"
urdf = joinpath(srcdir, "open_manipulator.urdf")
mechanism = parse_urdf(urdf)

In [ ]:
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf)); 

In [ ]:
set_configuration!(mvis, [0.5, 0.0, 0.0, 0.0, 0.0, 0.0])

In [ ]:
first_joint = findjoint(mechanism, "joint1")
transform_to_root(mvis.state, frame_after(first_joint))

We see that the poisiton after the first joint is the same as the relative position given in the URDF file. 

Now let's move on to the next joint. We have to calculate the position of the second joint after the first joint and the connecting link. We can do this by multiplying the transformation matrix of the first joint with the transformation matrix of the second joint.

In [ ]:
# example for link 2 - joint 2 - Rotation around y-axis

# Joint position (translation vector)
P_2 = [0.0, 0.0, 0.0595] # [x, y, z]

# Joint variable orientation (rotation matrix)
θ_2 = 0.0 # The current angle of the joint, note that the angles are in radians
R_variable_2 = # TODO: Create a rotation matrix from the angle θ around the y-axis

# Combine static and variable rotations
R_2 = # TODO: Combine the static and variable rotation matrices

# Homogeneous transformation matrix
T_2 = [R_2 P_2; 0 0 0 1]

Total_T = T * T_2

In [ ]:
second_joint = findjoint(mechanism, "joint2")
transform_to_root(mvis.state, frame_after(second_joint))

### Exercise: 
Check if our calculations are correct if we change the angles of the first joint. What do you expect to happen?

Let's also check the orientation. The orientation is the rotation part of the transformation matrix. We can extract the rotation part of the transformation matrix by taking the first three rows and columns of the transformation matrix and convert them back into rpy (roll, pitch, yaw) angles.

In [ ]:
function forward_kinematics_rpy(T0E)
    # Extract the rotation matrix from the transformation matrix
    R0E = T0E[1:3, 1:3]

    # Convert the rotation matrix to RPY angles
    roll = atan(R0E[3, 2] / R0E[3, 3])
    pitch = asin(-R0E[3, 1])
    yaw = atan(R0E[2, 1] / R0E[1, 1])

    return [roll, pitch, yaw]
end

In [ ]:
orientation = forward_kinematics_rpy(Total_T)

## Exercise: Implement Forward Kinematics

After we defined the transformation matrix for the first joints, we want to have a function that calculates the variable transformation matrix ```R_variable``` based on the joint's angle and the roation axis called ```RotationMatrix(θ::Number, axis::String)```.

In [ ]:
function RotationMatrix(θ::Number, axis::String)
    if axis == "z" # elseif  ... 
        # TODO: Create a rotation matrices from the angle θ around the given axis (x, y, z)
        # HINT: How to calculate the rotation matrix is given in the explanation above 
    end
    return R_variable
end

Now let us define a function ```get_tranformation_matrix(rel_pos::Vector{Float64}, θ::Float64)``` which takes relative position and rotation of the current joint and calculates the transformation matrix ```T``` for this joint. 

In [ ]:
function get_tranformation_matrix(rel_pos::Vector{Float64}, θ::Float64, axis::String)
    if axis=="None"
        R_variable = [1 0 0; 0 1 0; 0 0 1]
    else
        R_variable = RotationMatrix(θ, axis)
    end
    T = [R_variable rel_pos; 0 0 0 1]
    return T
end

Now let us define a function ```forward_kinematics``` that takes the joint angles as input and returns the transformation matrix of the end effector. 
The kinematic chain is as follows: 
    
    ```origin -> link1 -> joint1 -> link2 -> joint2 -> link3 -> joint3 -> link4 -> joint4 -> link5 -> gripper```

In [ ]:
function forward_kinematics(q)
    # TODO: Implement the forward kinematics for the OpenManipulatorBase
    # HINT: First estimate the specific transformation matrices for each joint 
    # and then combine them to get the transformation matrix from the base to the end-effector
    # You should have matrices for T01, T12, T23, T34, and T4E
    # HINT: Use the get_tranformation_matrix function
    return T0E
end

In [ ]:
T0E = forward_kinematics([0.6, 0.6, -0.5, -0.1])

Now we need to extract the position of the end effector from the transformation matrix. As seen before: the position of the end effector is the last column of the transformation matrix.

In [ ]:
position = T0E[1:3, 4]

Let's also check the orientation:

In [ ]:
orientation = forward_kinematics_rpy(T0E)

Putting it all together:

In [ ]:
function complete_forward_kinematics(q)
    T0E = forward_kinematics(q)
    position = T0E[1:3, 4]
    orientation = forward_kinematics_rpy(T0E)
    return [position, orientation]
end

In [ ]:
pos_heading = complete_forward_kinematics([0.6, 0.6, -0.5, -0.1])

## Evaluating our Implementation

Now lets evaluate our implementation by calculating the forward kinematics for the OpenManipulator-X using RigidBodyDynamics and comparing it to the our estimate of position and orientation.

In [ ]:
set_configuration!(mvis, [0.6, 0.6, -0.5, -0.1, 0.0, 0.0]) 

In [ ]:
state = mvis.state
joint_state = mvis.state.q

In [ ]:
pos_heading = complete_forward_kinematics([0.6, 0.6, -0.5, -0.1])

In [ ]:
gripper_joint = findjoint(mechanism, "gripper")
transform_to_root(mvis.state, frame_after(gripper_joint))

# Inverse Kinematics
In this section we will implement a method for the inverse kinematics. It will estimate the joint angles based on the position and orientation of the end effector. Today we will use the NLsolve library to solve the inverse kinematics problem. In later lectures we will learn how to implement our own inverse kinematics solver. 

In [ ]:
Pkg.add("NLsolve")
Pkg.add("LinearAlgebra")

In [ ]:
using NLsolve
using LinearAlgebra

To estimate the joint angles we first define a function that we want to solve. This function takes the joint angles as input and returns the difference between the current position and the desired position of the end effector. We want to find a point where this difference is zero. 

Therefore we set an initial guess and use the NLsolve library to solve the function.

In [ ]:
function inverse_kinematics(desired_pos)

    # Define the function to be solved
    function f!(F, q)
        # TODO: Implement the function for the inverse kinematics
        # HINT: First Calculate the forward kinematics for the current joint angles and get the position 
        # HINT: Then calculate the error between the current position and the desired position
        F[1] = ... 
        F[2] = ...
        F[3] = ...
    

    # Set an initial guess for the joint angles
    q0 = [0.0, 0.0, 0.0, 0.0]

    # Solve for the joint angles using NLsolve
    result = nlsolve(f!, q0)

    # Check if the solver converged
    if !result.f_converged
        println("Solver did not converge!")
        println(result)
        return nothing
    end

    # Return the joint angles
    println(result)
    return result.zero
end

Let's test our inverse kinematics function by giving it a desired position and seeing if it can find the joint angles that will move the end effector to that position. Let's use the position we estimated erlier.

In [ ]:
pos = pos_heading[1]

In [ ]:
inverse_kinematics(pos) # 0.6, 0.6, -0.5, -0.1

Well this is not the position we wanted. The reason for this is that the inverse kinematics solver finds a solution to the problem we provided. We did however not provide the heading of the end effector and therefore the solver found a solution where the end effector is pointing in a different direction than the one we wanted. 

Let us define a new ```inverse_kinematics``` function that takes the desired position and orientation as input and returns the joint angles that will move the end effector to that position and orientation. Since we already have a function that calculates the forward kinematics and gives us a position and orientation we can use that to calculate the current position and orientation of the end effector and set a fourth constraint for the inverse kinematics solver. Keep in mind that we only have 4 joints and therefore we can only set 4 constraints.

In [ ]:
function inverse_kinematics(desired_pos, desired_orient)
    # Define the function to be solved
    function inv_f!(F, q)
        # TODO: Implement the function for the inverse kinematics using the desired position and orientation
        # HINT: First Calculate the forward kinematics for the current joint angles and get the position and orientation
        # HINT: Then calculate the error between the current position and the desired position as before
        # HINT: Calculate the error between the current orientation and the desired orientation using the RPY angles
    end

    # Set an initial guess for the joint angles
    q0 = [0.0, 0.0, 0.0, 0.0]

    # Solve for the joint angles using NLsolve
    result = nlsolve(inv_f!, q0)

    # Check if the solver converged
    if !result.f_converged
        println("Solver did not converge")
        return nothing
    end

    # Return the joint angles
    println(result)
    return result.zero
end

Let's test our new inverse kinematics function by giving it a desired position and orientation and seeing if it can find the joint angles that will move the end effector to that position and orientation.

In [ ]:
pos = pos_heading[1]
heading = pos_heading[2]
inverse_kinematics(pos, heading) # 0.6, 0.6, -0.5, -0.1

That looks better. We can now use this method to move the end effector to any position and orientation we want.

However, there is one problem. The inverse kinematics solver can only find a solution if the desired position and orientation is reachable. If we give it a position and orientation that is not reachable it will not find a solution. Let's test this by giving it a position and orientation that is not reachable.

In [ ]:
inverse_kinematics([0.3, 0.3, 0.16], [0.0, 0.0, 0.0])

There is another Problem. The inverse kinematics solver can find a solution even if the robot itself is not able to reach the desired position. This may be for example due to constraints of the maximum rotation of the joints.

Let's test this by giving it a position that is reachable but the robot itself is not able to reach. We can find one by checking the limits of the joints and using the forward kinematic solver to find the position of the end effector for the maximum rotation of the joints.
We have the following limits: 

Joint 1:
```<limit velocity="4.8" effort="1" lower="${-pi*0.9}" upper="${pi*0.9}" />```

Joint 2:
```<limit velocity="4.8" effort="1" lower="${-pi*0.57}" upper="${pi*0.5}" />```

Joint 3:
```<limit velocity="4.8" effort="1" lower="${-pi*0.3}" upper="${pi*0.44}" />```

Joint 4:
```<limit velocity="4.8" effort="1" lower="${-pi*0.57}" upper="${pi*0.65}" />```

In [ ]:
pos_heading = complete_forward_kinematics([-π*1.2, 0.6, -0.5, -0.1]) # above limit for joint 1

In [ ]:
inverse_kinematics(pos_heading[1])

So we can see that we have to be careful with the results of our inverse kinematics solver. We have to check if the solution is reachable and if the robot itself is able to reach the desired position. 